# Khaerully Juanda 2602148480 - Quiz Computer Vision

# Import Library

In [10]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img, img_to_array
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

# Data Path

In [11]:
#masukin data path
TRAIN_DIR = 'train'
TEST_DIR = 'test'

# Buat Dataframe dengan Image paths & label

In [12]:
def createdataframe(dir):
    image_paths = []
    labels = []
    
    for item in os.listdir(dir  ):
        item_path = os.path.join(dir, item)
        
        if os.path.isdir(item_path):  # kalo directory, process images didalem
            for imagename in os.listdir(item_path):
                image_paths.append(os.path.join(item_path, imagename))
                labels.append(item)  # pake directory name jadi label
            print(f"Processed directory: {item}")
        elif os.path.isfile(item_path):  # kalo file e bener tinggal di add
            image_paths.append(item_path)
            labels.append("unknown")  # jadiin default label buat files sendiri
            print(f"Added file: {item_path}")
    
    return image_paths, labels

### DataFrames buat Training and Testing Data

In [13]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

Processed directory: angry
Processed directory: disgusted
Processed directory: fearful
Processed directory: happy
Processed directory: neutral
Processed directory: sad
Processed directory: surprised


In [14]:
print(train)

                          image      label
0         train\angry\im100.png      angry
1        train\angry\im1000.png      angry
2        train\angry\im1001.png      angry
3        train\angry\im1004.png      angry
4        train\angry\im1008.png      angry
...                         ...        ...
6431  train\surprised\im988.png  surprised
6432  train\surprised\im993.png  surprised
6433  train\surprised\im994.png  surprised
6434  train\surprised\im997.png  surprised
6435  train\surprised\im999.png  surprised

[6436 rows x 2 columns]


In [15]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

Added file: test\100424315.png
Added file: test\101474812.png
Added file: test\101505493.png
Added file: test\104363499.png
Added file: test\104595578.png
Added file: test\105030170.png
Added file: test\1053335.png
Added file: test\10535326.png
Added file: test\1053632.png
Added file: test\105454682.png
Added file: test\106454535.png
Added file: test\106494122.png
Added file: test\109424843.png
Added file: test\110303600.png
Added file: test\110535360.png
Added file: test\110555847.png
Added file: test\111232888.png
Added file: test\111545756.png
Added file: test\111676739.png
Added file: test\11303133.png
Added file: test\113222858.png
Added file: test\1132334.png
Added file: test\114343625.png
Added file: test\1144048.png
Added file: test\114545394.png
Added file: test\115383273.png
Added file: test\115414368.png
Added file: test\115515163.png
Added file: test\116464963.png
Added file: test\116535998.png
Added file: test\117040844.png
Added file: test\118070572.png
Added file: test\1

In [16]:
print(test)
print(test['image'])

                   image    label
0     test\100424315.png  unknown
1     test\101474812.png  unknown
2     test\101505493.png  unknown
3     test\104363499.png  unknown
4     test\104595578.png  unknown
...                  ...      ...
1432  test\996343125.png  unknown
1433  test\997363900.png  unknown
1434   test\99936359.png  unknown
1435  test\999404971.png  unknown
1436     test\Answer.csv  unknown

[1437 rows x 2 columns]
0       test\100424315.png
1       test\101474812.png
2       test\101505493.png
3       test\104363499.png
4       test\104595578.png
               ...        
1432    test\996343125.png
1433    test\997363900.png
1434     test\99936359.png
1435    test\999404971.png
1436       test\Answer.csv
Name: image, Length: 1437, dtype: object


# Buat CNN Model

In [20]:
#pake CNN keras
import tensorflow as tf
from tensorflow.keras import layers, models

def create_cnn_model(input_shape, num_classes):
    model = models.Sequential()

    # Convolutional layer
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    
    # Max pooling layer
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    # Flatten the output for the dense layer
    model.add(layers.Flatten())
    
    # Dense layer
    model.add(layers.Dense(64, activation='relu'))
    
    # Dropout layer
    model.add(layers.Dropout(0.5))
    
    # Output layer
    model.add(layers.Dense(num_classes, activation='softmax'))  # pake 'sigmoid' buat binary klasifikasi
    
    return model

# Model Compilation
input_shape = (48, 48, 3)  # ngubah image size sama channels
num_classes = 10  # number of classes

model = create_cnn_model(input_shape, num_classes)

# Compile model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # gunain 'binary_crossentropy' for binary classification
              metrics=['accuracy'])

# Print ringkasan dari modelnya 
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (None, 46, 46, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 16928)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │     1,083,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,085,002 (4.14 MB)

 Trainable params: 1,085,002 (4.14 MB)

 Non-trainable params: 0 (0.00 B)

# Buat image preprocessing function

In [21]:
# Buat image preprocessing function
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(48, 48))
    img_array = img_to_array(img) / 255.0  # Normalize gambar
    img_array = np.expand_dims(img_array, axis=0)  
    return img_array

# Predict & update ke Answer.csv

In [23]:
def predict_and_update(csv_path, image_folder):
    # Load CSV nya
    df = pd.read_csv(csv_path)

    if 'Label' not in df.columns:
        df['Label'] = None

    # looping gambar testnya
    for index, row in df.iterrows():
        image_name = row['Image_Name']
        image_path = os.path.join(image_folder, image_name)

        if os.path.exists(image_path):
            img_array = preprocess_image(image_path)
            prediction = model.predict(img_array)
            predicted_class = np.argmax(prediction, axis=1)[0]
            df.at[index, 'Label'] = int(predicted_class)
        else:
            print(f"Image not found: {image_name}")

    # nyimpen dataframe ke csv
    df.to_csv(csv_path, index=False)
    print(f"Predictions updated in {csv_path}")

# function buat compare ke csv (label)
csv_path = 'Answer.csv'  # Update ke Answer.csv 
image_folder = 'test'  # update & bandingin sama gambar test

predict_and_update(csv_path, image_folder)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━